In [21]:
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

In [3]:
import pickle as pkl

In [31]:
import pandas as pd

# Mis features

In [9]:
data = pkl.load(open('features_juli_with_support.pkl', 'rb'))

In [10]:
data.columns

Index(['visited site', 'generic listing', 'conversion', 'viewed product',
       'person', 'tiempo_promedio_por_dia', 'label', 'modelo_mas_visto',
       'condition_mode', 'storage_mode', 'campaing_mode', 'cant_model_vistos',
       'event_support'],
      dtype='object')

In [11]:
data.shape

(19414, 13)

In [12]:
clf = Perceptron(tol=1e-3, random_state=0)

In [14]:
X_train = data[['visited site', 'generic listing', 'conversion', 'viewed product',
        'tiempo_promedio_por_dia','modelo_mas_visto',
       'condition_mode', 'storage_mode', 'campaing_mode', 'cant_model_vistos',
       'event_support']]

In [16]:
y_train = data.label

In [19]:
#split del dataset para entrenamiento y pruebas, 80% y 20% respectivamente
Xtrn, Xtest, Ytrn, Ytest = train_test_split(X_train,y_train,test_size=0.2)

In [20]:
clf.fit(Xtrn, Ytrn)

Perceptron(alpha=0.0001, class_weight=None, early_stopping=False, eta0=1.0,
      fit_intercept=True, max_iter=None, n_iter=None, n_iter_no_change=5,
      n_jobs=None, penalty=None, random_state=0, shuffle=True, tol=0.001,
      validation_fraction=0.1, verbose=0, warm_start=False)

In [22]:
roc_auc_score(Ytest, clf.predict(Xtest))

0.5021326712503184

# Features Chicas

In [23]:
dataC = pkl.load(open('features_chicas.pkl','rb'))

In [26]:
dataC['device_frecuente'].fillna(dataC['device_frecuente'].mode()[0], inplace=True)

In [27]:
dataC[ 'Returning'].fillna(dataC['Returning'].mode()[0], inplace=True)

In [28]:
import category_encoders as ce

In [29]:
te_channel_frecuente = ce.TargetEncoder(handle_unknown='ignore')

In [32]:
label_train = pd.read_csv('data/fiuba-trocafone-tp2-final-set/labels_training_set.csv')

In [34]:
trainC = pd.merge(dataC, label_train, on='person', how='right')

In [35]:
trainC['Returning'].fillna(trainC['Returning'].mode()[0], inplace=True)

In [36]:
trainC['device_frecuente'].fillna(trainC['device_frecuente'].mode()[0], inplace=True)

In [38]:
xTrainC = trainC[[ 'cant_eventos_totales', 'cant_ach', 'cant_bl', 'cant_ch',
       'cant_cv', 'cant_gl', 'cant_l', 'cant_seh', 'cant_s_p', 'cant_sp',
       'cant_vp', 'cant_vs', 'Returning', 'tiempo_total',
       'tiempo_new_returning', 'channel_frecuente', 'device_frecuente', 'q1',
       'q2', 'q3', 'q4', 'q5', 'q6', 'q7', 'q8', 'q9', 'q10']]

In [39]:
yTrainC = trainC.label

In [40]:
xTrainC.channel_frecuente = te_channel_frecuente.fit_transform(pd.DataFrame(xTrainC.channel_frecuente), yTrainC)

/home/july/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:3110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [41]:
te_device_frecuente = ce.TargetEncoder(handle_unknown='ignore')

In [42]:
xTrainC.device_frecuente = te_device_frecuente.fit_transform(pd.DataFrame(xTrainC.device_frecuente), yTrainC)

/home/july/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:3110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [43]:
#split del dataset para entrenamiento y pruebas, 80% y 20% respectivamente
Xtrn2, Xtest2, Ytrn2, Ytest2 = train_test_split(xTrainC,yTrainC,test_size=0.2)

In [44]:
clf.fit(Xtrn2,Ytrn2)

Perceptron(alpha=0.0001, class_weight=None, early_stopping=False, eta0=1.0,
      fit_intercept=True, max_iter=None, n_iter=None, n_iter_no_change=5,
      n_jobs=None, penalty=None, random_state=0, shuffle=True, tol=0.001,
      validation_fraction=0.1, verbose=0, warm_start=False)

In [45]:
roc_auc_score(Ytest2, clf.predict(Xtest2))

0.5039201901254002

In [47]:
sum(clf.predict(Xtest2) == 1)

443

# Perceptron no nos ofrece probabilidades.